# Data Analysis

<p>This file is use to analyse the data files generated using the OpenStuido software as a part of DMX9213 research. The below steps will be followed for the data analysis.</p>

- Step 1: Open .csv file and preprocess the data.
- Step 2: Calculate the PMV values for each hour.
- Step 3: Identify the month with highest cumilative PMV values averaged over 30 days from the baselins.
- Step 4: Check if the months are different from each baselines. If different, use the month that selected by majority of models.
- Step 5: Average the temperature, humidity and PMV values for the selected month. Average values should represent 24 hour variation of temperature and humidity for the selected month.
- Step 6: Generate graphs to compair the temperature, humidity and PMV values for the baseline and the passive cooling implimented models. The comparison should be performend for the selected month.


### General

This section will be used to import the data necessary for all simulation.

In [1]:
%%time 

# install packages
%pip install pandas
%pip install matplotlib
%pip install ipywidgets
%pip install ipympl

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.
CPU times: total: 15.6 ms
Wall time: 5.61 s



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
# import packages
import math
import pandas as pd
import os
import matplotlib.pyplot as plt
import numpy as np
import ipywidgets
from IPython.display import display

%matplotlib widget

C:\Users\ROG\AppData\Local\Temp\ipykernel_24368\1941769566.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
# paths
BASELINE_PATH = "input/baseline/"
IMPROVED_PATH = "input/improved/"
baseline_csv_files = os.listdir(BASELINE_PATH)
improved_csv_files = os.listdir(IMPROVED_PATH)

In [4]:
# import the test file
TEST_FILE_PATH = "input/test.csv"

In [5]:
# parameter to run the test cells
run_test = False

### Step 1


Open .csv file and preprocess the data.

In [6]:
def preprocess_data(file_path: str) -> pd.DataFrame:

    df = pd.read_csv(file_path)

    # remove columns relevent to plenum space
    columns_to_drop = [column_name for column_name in df.columns if "PLENUM" in column_name]
    df.drop(columns=columns_to_drop, inplace=True)

    # rename the columns
    for column_name in df.columns:
        df.rename(columns={column_name: column_name.replace('LIVINGTHERMALZONE:Zone', '').strip()}, inplace=True)

    # change the hourly column to datetime
    df["Hourly"] = pd.to_datetime(df["Hourly"])

    return df

In [7]:
if run_test:
    step_1_test_df = preprocess_data(TEST_FILE_PATH)
    display(step_1_test_df.head())

### Step 2

Calculate PMV values for each hour. \
Refer to following link for more details on PMV calculation. \
https://github.com/chathusha/PmvCalculator

In [8]:
def calculate_clo(m: float, icl: float) -> float:
    """
    Calculates the clo for a given metabolic rate and clothing insulation using ASHRAE 55:2023.
    
    Args:
        m (float): metabolic rate in met unit
        icl (float): clothing insulation in clo unit
    
    Return:
        float: clo value to use in the PMV calculator
    """
    if m < 1.2:
        clo = icl

    else:
        clo = icl * (0.6 + 0.4 / m)

    return clo

In [9]:
def calculate_pmv(ta: float, tr: float, vel: float, rh: float, met: float, clo: float, wme: float) -> dict:
    """
     Calculate the PMV for given inputs using ASHRAE 55:2023. The air speed shouldn't be above 0.1 m/s.

     Args:
          ta (float): air temperature in C
          tr (float): mean radiant temperature in C
          vel(float): air speed in m/s
          rh (float): relative humidity in %
          met (float): metabolic rate in met unit
          clo (float): clothing factor in clo unit (calculated using calculate_clo method)
          wme (float): external work in met

     Return:
          dict: {
               pmv: Predicted Mean Vote (PMV),
               ppd: Predicted Precentage of Dissatisfied (PPD),
               status: status of the calculation, 0 succeed, 1 error
               }
     """
    if met > 1:
        pa = rh * 10. * math.exp(16.6536 - 4030.183 / (ta + 235.))
        icl = 0.155 * clo  # thermal insulation of the clothing in m2K/W
        m = met * 58.15  # metabolic rate in W/m2
        w = wme * 58.15  # extenal work in W/m2
        mw = m - w  # internal heat production in the humen body

        if icl <= 0.078:
            fcl = 1 + (1.29 * icl)
        else:
            fcl = 1.05 + (0.645 * icl)

        # heat transfer coefficient by forced convection
        hcf = 12.1 * math.sqrt(vel)
        taa = ta + 273.
        tra = tr + 273.
        tcla = taa + (35.5 - ta) / (3.5 * icl + 0.1)
        p1 = icl * fcl
        p2 = p1 * 3.96
        p3 = p1 * 100.
        p4 = p1 * taa
        p5 = 308.7 - 0.028 * mw + (p2 * math.pow(tra / 100, 4))
        xn = tcla / 100.
        xf = tcla / 50.
        eps = 0.00015
        n = 0

        while abs(xn - xf) > eps:
            xf = (xf + xn) / 2
            hcn = 2.38 * math.pow(abs(100. * xf - taa), 0.25)
            hc = hcf if hcf > hcn else hcn
            xn = (p5 + p4 * hc - p2 * math.pow(xf, 4)) / (100. + p3 * hc)
            n += 1
            if n > 150:
                print("Max iterations exceeded")
                return {
                    'pmv': 0,
                    'ppd': 0,
                    'status': 1
                    }

        tcl = 100. * xn - 273.

        # heat loss diff. through skin
        hl1 = 3.05 * 0.001 * (5733 - (6.99 * mw) - pa)

        # heat loss by sweating
        hl2 = 0.42 * (mw - 58.15) if (mw > 58.15) else 0

        # latent respiration heat loss
        hl3 = 1.7 * 0.00001 * m * (5867. - pa)

        # dry respiration heat loss
        hl4 = 0.0014 * m * (34. - ta)

        # heat loss by radiation
        hl5 = 3.96 * fcl * (math.pow(xn, 4) - math.pow(tra / 100., 4))

        # heat loss by convection
        hl6 = fcl * hc * (tcl - ta)

        ts = 0.303 * math.exp(-0.036 * m) + 0.028
        pmv = ts * (mw - hl1 - hl2 - hl3 - hl4 - hl5 - hl6)
        ppd = 100 - 95 * \
        math.exp(-0.03353 * math.pow(pmv, 4) - 0.2179 * math.pow(pmv, 2))

        return {
            'pmv': pmv,
            'ppd': ppd,
            'status': 0
            }

    else:
        print("ASHRAE 55:2023 calculation does not available for metabolic rate less than 1")
        return {
            'pmv': 0,
            'ppd': 0,
            'status': 1
            }

In [10]:
def add_pmv_column(df: pd.DataFrame) -> pd.DataFrame:

    df['PMV'] = df.apply(lambda x: calculate_pmv(
        ta=x['Air Temperature[C]'],
        tr=x['Mean Radiant Temperature[C]'],
        vel=0.1,
        rh=x['Air Relative Humidity[%]'],
        met=1.2,
        clo=calculate_clo(m=1.2, icl=0.36),
        wme=0
        )['pmv'], axis=1)

    return df

In [11]:
if run_test:
    step_2_test_df = add_pmv_column(step_1_test_df)
    display(step_2_test_df.head())

### Step 3

Calculate the average pmv value for each month. \
Identify the month with highest cumilative PMV values averaged over 30 days from the baseline.

In [12]:
def get_month_with_highest_cum_pmv(df: pd.DataFrame) -> int:

    # calculate the mean pmv for day of each month
    group_by_month = df.groupby(df["Hourly"].dt.month)["PMV"].mean()*24

    # multiply the value by 30 to get monthly avearage
    group_by_month = group_by_month * 30

    max_cum_pmv_month = group_by_month.index[group_by_month == group_by_month.max()][0]
    
    return max_cum_pmv_month

In [13]:
if run_test:
    step_3_test = get_month_with_highest_cum_pmv(step_2_test_df)
    display(step_3_test)

### Step 4

Calculate the PMV value for all building models in the baseline.\
Check if the month with highest cumilative PMV values averaged over 30 days from the baseline is different from each baselines.

In [14]:
months_with_highest_pmv = {}
baseline_dfs = {}

for baseline_csv_file in baseline_csv_files[:4 if run_test else len(baseline_csv_files)]:
    preprocessed_baseline_df = preprocess_data(os.path.join(BASELINE_PATH, baseline_csv_file))
    baseline_df = add_pmv_column(preprocessed_baseline_df)
    max_month_with_cum_pmv = get_month_with_highest_cum_pmv(baseline_df)
    months_with_highest_pmv[baseline_csv_file] = max_month_with_cum_pmv

    # store baseline files
    baseline_dfs[baseline_csv_file.strip(".csv")] = baseline_df

months_with_highest_cum_pmv_df = pd.DataFrame(
    months_with_highest_pmv.items(),
    columns=['File Name', 'Month with Highest Cumulative PMV']
)

display(months_with_highest_cum_pmv_df)


,File Name,Month with Highest Cumulative PMV
0,10deg_roof_without_ceiling_0deg_no_roof_paint.csv,4
1,10deg_roof_without_ceiling_180deg_no_roof_pain...,4
2,10deg_roof_without_ceiling_270deg_no_roof_pain...,4
3,10deg_roof_without_ceiling_90deg_no_roof_paint...,4
4,10deg_roof_with_ceiling_0deg_no_roof_paint.csv,4
5,10deg_roof_with_ceiling_180deg_no_roof_paint.csv,4
6,10deg_roof_with_ceiling_270deg_no_roof_paint.csv,4
7,10deg_roof_with_ceiling_90deg_no_roof_paint.csv,4
8,15deg_roof_without_ceiling_0deg_no_roof_paint.csv,4
9,15deg_roof_without_ceiling_180deg_no_roof_pain...,4


In [30]:
# save the results into a csv file
months_with_highest_cum_pmv_df.to_csv(".\\output\\months_with_highest_cum_pmv.csv", index=False)

In [16]:
# get the most common month from above data
month_with_highest_cum_pmv = months_with_highest_cum_pmv_df['Month with Highest Cumulative PMV'].mode()[0]
print(f"Most common month with highest cumulative PMV: {month_with_highest_cum_pmv}")

Most common month with highest cumulative PMV: 4


### Step 5

Calculate the hourly averaged temperature, humidity and PMV values for the selected month using baseline data.

In [17]:
processed_baseline_dfs = {}

for key, baseline_df in baseline_dfs.items():
    df = baseline_df.loc[baseline_df["Hourly"].dt.month == month_with_highest_cum_pmv]

    # create columns for month and hour
    selected_month_df = df.copy()
    selected_month_df["Month"] = df["Hourly"].dt.month.astype(int)
    selected_month_df["Hour"] = df["Hourly"].dt.hour.astype(int)
    

    # drop hourly column
    selected_month_df.drop(columns=["Hourly"], inplace=True)
    
    # calculate the hourly averaged the temperature, humidity and PMV
    selected_month_df = selected_month_df.groupby(selected_month_df["Hour"]).mean()
    selected_month_df.reset_index(inplace=True)

    processed_baseline_dfs[key] = selected_month_df

In [18]:
if run_test:
    display(processed_baseline_dfs[list(processed_baseline_dfs.keys())[0]])

### Step 6

Convert the improved csv files to processed dataframes.\
Generate graphs to compair the temperature, humidity and PMV values for the baseline and the passive cooling implimented models.

In [19]:
processed_improved_dfs = {}

for improved_csv_file in improved_csv_files[0:4 if run_test else len(improved_csv_files)]:
    preprocessed_improved_df = preprocess_data(os.path.join(IMPROVED_PATH, improved_csv_file))
    improved_df = add_pmv_column(preprocessed_improved_df)

    # seperate data only relevent to selected month
    df = improved_df.loc[improved_df["Hourly"].dt.month == month_with_highest_cum_pmv]

    # create columns for month and hour
    highest_cum_pmv_month_df = df.copy()
    highest_cum_pmv_month_df["Month"] = df["Hourly"].dt.month.astype(int)
    highest_cum_pmv_month_df["Hour"] = df["Hourly"].dt.hour.astype(int)

    # drop hourly column
    highest_cum_pmv_month_df.drop(columns=["Hourly"], inplace=True)

    # calculate the hourly averaged the temperature, humidity and PMV
    highest_cum_pmv_month_df = highest_cum_pmv_month_df.groupby(highest_cum_pmv_month_df["Hour"]).mean()
    highest_cum_pmv_month_df.reset_index(inplace=True)

    processed_improved_dfs[improved_csv_file.strip(".csv")] = highest_cum_pmv_month_df
    

In [20]:
if run_test:
    df = processed_baseline_dfs[list(processed_baseline_dfs.keys())[0]]
    plt.figure(figsize=(6, 4), dpi=200)
    plt.plot('Hour', 'Air Temperature[C]', data=df, label='Room', linewidth=1.0)
    plt.plot('Hour', 'Outdoor Air Drybulb Temperature[C]', data=df, label='Outdoor', linewidth=1.0, linestyle='-.')

    plt.title('Hourly Temperature Variation')
    plt.xlabel('Hour')
    plt.ylabel('Temperature (C)')

    plt.xticks(np.arange(0, 24, 1))

    plt.xlim(0, 23)
    plt.ylim(24, 40)

    plt.grid(True)
    plt.legend(loc='upper left')
    plt.show()

#### Compare the building orientation effect on room temperature

In [21]:
def plot_orientation_temp_change(roof_angle, ceiling):
    base_file_name = f"{str(roof_angle)+'deg' if roof_angle != 0 else 'flat'}_roof_{ceiling}"

    df_0 = processed_baseline_dfs[f"{base_file_name}_0deg_no_roof_paint"]
    df_90 = processed_baseline_dfs[f"{base_file_name}_90deg_no_roof_paint"]
    df_180 = processed_baseline_dfs[f"{base_file_name}_180deg_no_roof_paint"]
    df_270 = processed_baseline_dfs[f"{base_file_name}_270deg_no_roof_paint"]

    plt.figure(figsize=(6, 4), dpi=200)
    plt.plot('Hour', 'Air Temperature[C]', data=df_0, label='0 degree', linewidth=1.0)
    plt.plot('Hour', 'Air Temperature[C]', data=df_90, label='90 degree', linewidth=1.0)
    plt.plot('Hour', 'Air Temperature[C]', data=df_180, label='180 degree', linewidth=1.0)
    plt.plot('Hour', 'Air Temperature[C]', data=df_270, label='270 degree', linewidth=1.0)

    plt.title('Hourly Temperature Variation')
    plt.xlabel('Hour')
    plt.ylabel('Temperature (C)')

    plt.xticks(np.arange(0, 24, 1))

    plt.xlim(0, 23)
    plt.ylim(24, 40)

    plt.grid(True)
    plt.legend(loc='upper left')
    plt.show()

In [22]:
ipywidgets.interact(plot_orientation_temp_change, roof_angle=[0, 5, 10, 15, 20], ceiling=['without_ceiling', 'with_ceiling'])

interactive(children=(Dropdown(description='roof_angle', options=(0, 5, 10, 15, 20), value=0), Dropdown(descri…

<function __main__.plot_orientation_temp_change(roof_angle, ceiling)>

#### Compare the effect of roof angle on room temperature

In [23]:
def plot_roof_angle_temp_change(orientation, ceiling, roof_paint):
    base_file_name = f"_roof_{ceiling}_{orientation}deg_{roof_paint}"

    if roof_paint == 'roof_paint':
        df_0_deg_roof = processed_improved_dfs[f"flat{base_file_name}"]
        df_5_deg_roof = processed_improved_dfs[f"5deg{base_file_name}"]
        df_10_deg_roof = processed_improved_dfs[f"10deg{base_file_name}"]
        df_15_deg_roof = processed_improved_dfs[f"15deg{base_file_name}"]
        df_20_deg_roof = processed_improved_dfs[f"20deg{base_file_name}"]
    else:
        df_0_deg_roof = processed_baseline_dfs[f"flat{base_file_name}"]
        df_5_deg_roof = processed_baseline_dfs[f"5deg{base_file_name}"]
        df_10_deg_roof = processed_baseline_dfs[f"10deg{base_file_name}"]
        df_15_deg_roof = processed_baseline_dfs[f"15deg{base_file_name}"]
        df_20_deg_roof = processed_baseline_dfs[f"20deg{base_file_name}"]

    plt.figure(figsize=(6, 4), dpi=200)
    plt.plot('Hour', 'Air Temperature[C]',
             data=df_0_deg_roof, label='0 degree roof', linewidth=1.0)
    plt.plot('Hour', 'Air Temperature[C]',
             data=df_5_deg_roof, label='5 degree roof', linewidth=1.0)
    plt.plot('Hour', 'Air Temperature[C]',
             data=df_10_deg_roof, label='10 degree roof', linewidth=1.0)
    plt.plot('Hour', 'Air Temperature[C]',
             data=df_15_deg_roof, label='15 degree roof', linewidth=1.0)
    plt.plot('Hour', 'Air Temperature[C]',
             data=df_20_deg_roof, label='20 degree roof', linewidth=1.0)

    plt.title('Hourly Temperature Variation')
    plt.xlabel('Hour')
    plt.ylabel('Temperature (C)')

    plt.xticks(np.arange(0, 24, 1))

    plt.xlim(0, 23)
    plt.ylim(24, 40)

    plt.grid(True)
    plt.legend(loc='upper left')
    plt.show()

In [24]:
ipywidgets.interact(plot_roof_angle_temp_change, 
                    orientation=[0, 90, 180, 270], 
                    ceiling=['without_ceiling', 'with_ceiling'],
                    roof_paint=['no_roof_paint', 'roof_paint'])

interactive(children=(Dropdown(description='orientation', options=(0, 90, 180, 270), value=0), Dropdown(descri…

<function __main__.plot_roof_angle_temp_change(orientation, ceiling, roof_paint)>

#### Compare the room temperature variation before and after passive cooling

In [25]:
def plot_temp_change_after_improvement(roof_angle, ceiling, orientation):
    base_file_name = f"{str(roof_angle)+'deg' if roof_angle !=0 else 'flat'}_roof_{ceiling}_{orientation}deg"

    baseline_df = processed_baseline_dfs[f"{base_file_name}_no_roof_paint"]
    improved_df = processed_improved_dfs[f"{base_file_name}_roof_paint"]

    plt.figure(figsize=(6, 4), dpi=200)
    plt.plot('Hour', 'Air Temperature[C]', data=baseline_df, label='No roof paint', linewidth=1.0)
    plt.plot('Hour', 'Air Temperature[C]', data=improved_df, label='With roof paint', linewidth=1.0, linestyle='--')
    plt.plot('Hour', 'Outdoor Air Drybulb Temperature[C]', data=baseline_df, label='Outdoor', linewidth=1.0, linestyle='-.')

    plt.title('Hourly Temperature Variation')
    plt.xlabel('Hour')
    plt.ylabel('Temperature (C)')

    plt.xticks(np.arange(0, 24, 1))

    plt.xlim(0, 23)
    plt.ylim(24, 40)

    plt.grid(True)
    plt.legend(loc='upper left')
    plt.show()

In [26]:
ipywidgets.interact(plot_temp_change_after_improvement, 
                    roof_angle=[0, 5, 10, 15, 20], 
                    ceiling=['without_ceiling', 'with_ceiling'],
                    orientation=[0, 90, 180, 270]
                    )

interactive(children=(Dropdown(description='roof_angle', options=(0, 5, 10, 15, 20), value=0), Dropdown(descri…

<function __main__.plot_temp_change_after_improvement(roof_angle, ceiling, orientation)>

In [27]:
def plot_pmv_change_after_improvement(roof_angle, ceiling, orientation):
    base_file_name = f"{str(roof_angle)+'deg' if roof_angle !=
                        0 else 'flat'}_roof_{ceiling}_{orientation}deg"

    baseline_df = processed_baseline_dfs[f"{base_file_name}_no_roof_paint"]
    improved_df = processed_improved_dfs[f"{base_file_name}_roof_paint"]

    plt.figure(figsize=(6, 4), dpi=200)
    plt.plot('Hour', 'PMV', data=baseline_df,
             label='No roof paint', linewidth=1.0)
    plt.plot('Hour', 'PMV', data=improved_df,
             label='With roof paint', linewidth=1.0, linestyle='--')

    plt.title('Hourly PMV Variation')
    plt.xlabel('Hour')
    plt.ylabel('PMV')

    plt.xticks(np.arange(0, 24, 1))

    plt.xlim(0, 23)
    plt.ylim(-1, 4)

    plt.fill_between(x=range(24), y1=-0.5, y2=0.5, color='green', alpha=0.2, linewidth=0)
    plt.grid(True)
    plt.legend(loc='upper left')
    plt.show()

In [28]:
ipywidgets.interact(plot_pmv_change_after_improvement,
                    roof_angle=[0, 5, 10, 15, 20],
                    ceiling=['without_ceiling', 'with_ceiling'],
                    orientation=[0, 90, 180, 270]
                    )

interactive(children=(Dropdown(description='roof_angle', options=(0, 5, 10, 15, 20), value=0), Dropdown(descri…

<function __main__.plot_pmv_change_after_improvement(roof_angle, ceiling, orientation)>